In [1]:
import pandas as pd
import numpy as np

try:
    df_flight = pd.read_csv("Flight Level Data.csv")
except FileNotFoundError:
    print("Error: 'Flight Level Data.csv' not found. Please check your file paths.")
    exit()


df_flight['scheduled_departure_datetime_local'] = pd.to_datetime(df_flight['scheduled_departure_datetime_local'], utc=True)
df_flight['actual_departure_datetime_local'] = pd.to_datetime(df_flight['actual_departure_datetime_local'], utc=True)

df_flight['departure_delay_minutes'] = (
    df_flight['actual_departure_datetime_local'] - df_flight['scheduled_departure_datetime_local']
).dt.total_seconds() / 60

total_flights = df_flight.shape[0]

average_delay = df_flight['departure_delay_minutes'].mean()

delayed_flights_count = df_flight[df_flight['departure_delay_minutes'] > 0].shape[0]
percentage_delayed = (delayed_flights_count / total_flights) * 100


data = {
    'Metric': [
        'Average Departure Delay',
        'Flights Departing Late (Delay > 0)'
    ],
    'Value': [
        f"{average_delay:.2f}",
        f"{percentage_delayed:.2f}%"
    ],
    'Unit/Context': [
        'Minutes',
        'Percentage of All Flights'
    ]
}

df_output_q1 = pd.DataFrame(data)

print("### EDA Q1: Departure Delay Summary")
print(df_output_q1.to_markdown(index=False))

df_flight[['company_id', 'flight_number', 'scheduled_departure_date_local', 'departure_delay_minutes', 'scheduled_ground_time_minutes', 'minimum_turn_minutes']].to_csv('temp_df_flight_metrics.csv', index=False)

### EDA Q1: Departure Delay Summary
| Metric                             | Value   | Unit/Context              |
|:-----------------------------------|:--------|:--------------------------|
| Average Departure Delay            | 21.18   | Minutes                   |
| Flights Departing Late (Delay > 0) | 49.61%  | Percentage of All Flights |


In [2]:
import pandas as pd
import numpy as np

# Load the flight metrics calculated in the previous step
try:
    df_metrics = pd.read_csv('temp_df_flight_metrics.csv')
except FileNotFoundError:
    print("Error: 'temp_df_flight_metrics.csv' not found. Please run Code Block 1 first.")
    exit()


df_metrics['ground_time_buffer'] = df_metrics['scheduled_ground_time_minutes'] - df_metrics['minimum_turn_minutes']

total_flights = df_metrics.shape[0]

tight_ground_time_count = df_metrics[df_metrics['ground_time_buffer'] <= 0].shape[0]
percentage_tight_ground_time = (tight_ground_time_count / total_flights) * 100

very_tight_ground_time_count = df_metrics[df_metrics['ground_time_buffer'] <= 5].shape[0]
percentage_very_tight_ground_time = (very_tight_ground_time_count / total_flights) * 100


data = {
    'Metric': [
        'Flights with Scheduled Ground Time  Minimum Turn Mins',
        'Flights with Scheduled Ground Time (Min Turn + 5 Mins)'
    ],
    'Value': [
        f"{percentage_tight_ground_time:.2f}%",
        f"{percentage_very_tight_ground_time:.2f}%"
    ],
    'Count': [
        tight_ground_time_count,
        very_tight_ground_time_count
    ]
}

df_output_q2 = pd.DataFrame(data)

print("### EDA Q2: Ground Time Tightness Summary")
print(df_output_q2.to_markdown(index=False))

# Note: No new metrics are generated that need to be saved for the next step.

### EDA Q2: Ground Time Tightness Summary
| Metric                                                 | Value   |   Count |
|:-------------------------------------------------------|:--------|--------:|
| Flights with Scheduled Ground Time  Minimum Turn Mins  | 8.05%   |     652 |
| Flights with Scheduled Ground Time (Min Turn + 5 Mins) | 9.63%   |     780 |


In [3]:
import pandas as pd
import numpy as np

# Load the Bag Level Data
try:
    df_bag = pd.read_csv("Bag+Level+Data.csv")
except FileNotFoundError:
    print("Error: 'Bag+Level+Data.csv' not found. Please check your file paths.")
    exit()

# --- Calculations ---

df_bag['transfer_ind'] = df_bag['bag_type'].apply(
    lambda x: 1 if x in ['Transfer', 'Hot Transfer'] else 0
)
df_bag['origin_ind'] = df_bag['bag_type'].apply(lambda x: 1 if x == 'Origin' else 0)

flight_key = ['company_id', 'flight_number', 'scheduled_departure_date_local']
df_bag_agg = df_bag.groupby(flight_key).agg(
    transfer_bags=('transfer_ind', 'sum'),
    origin_bags=('origin_ind', 'sum')
).reset_index()

df_bag_agg['transfer_to_origin_ratio'] = df_bag_agg.apply(
    lambda row: row['transfer_bags'] / row['origin_bags'] if row['origin_bags'] > 0 else np.nan,
    axis=1
)

average_bag_ratio = df_bag_agg['transfer_to_origin_ratio'].mean()

# --- Tabular Output ---

data = {
    'Metric': [
        'Average Ratio of Transfer Bags vs. Origin Bags'
    ],
    'Value': [
        f"{average_bag_ratio:.4f}"
    ],
    'Unit/Context': [
        'Ratio (Transfer Bags : Origin Bags)'
    ]
}

df_output_q3 = pd.DataFrame(data)

print("### EDA Q3: Baggage Complexity Summary")
print(df_output_q3.to_markdown(index=False))

# Save the bag aggregation data for use in the final difficulty score calculation
df_bag_agg.to_csv('temp_df_bag_agg.csv', index=False)

### EDA Q3: Baggage Complexity Summary
| Metric                                         |   Value | Unit/Context                        |
|:-----------------------------------------------|--------:|:------------------------------------|
| Average Ratio of Transfer Bags vs. Origin Bags |  3.7458 | Ratio (Transfer Bags : Origin Bags) |


In [4]:
import pandas as pd
import numpy as np

# --- 1. Load Data ---
try:
    # Load metrics from previous step (Q1)
    df_metrics = pd.read_csv('temp_df_flight_metrics.csv')
    # Load PNR data for passenger counts
    df_pnr_flight = pd.read_csv("PNR+Flight+Level+Data.csv")
    # Load Flight Level Data again for total_seats, as it wasn't saved in temp_df_flight_metrics.csv in Q1
    # Re-loading df_flight directly is safer here to ensure all columns (like total_seats) are present.
    df_flight = pd.read_csv("Flight Level Data.csv")

except Exception as e:
    print(f"Error loading files: {e}")
    exit()

flight_key_cols = ['company_id', 'flight_number', 'scheduled_departure_date_local']

pax_key_cols = flight_key_cols + ['record_locator']
df_pax = df_pnr_flight.groupby(pax_key_cols)['total_pax'].max().reset_index()
df_total_pax = df_pax.groupby(flight_key_cols)['total_pax'].sum().reset_index(name='total_pax_booked')

df_merged_q4 = df_flight.merge(df_total_pax, on=flight_key_cols, how='left')

df_merged_q4['scheduled_departure_datetime_local'] = pd.to_datetime(df_merged_q4['scheduled_departure_datetime_local'], utc=True)
df_merged_q4['actual_departure_datetime_local'] = pd.to_datetime(df_merged_q4['actual_departure_datetime_local'], utc=True)
df_merged_q4['departure_delay_minutes'] = (df_merged_q4['actual_departure_datetime_local'] - df_merged_q4['scheduled_departure_datetime_local']).dt.total_seconds() / 60


df_merged_q4['total_pax_booked'] = df_merged_q4['total_pax_booked'].fillna(0)
df_merged_q4['load_factor'] = (df_merged_q4['total_pax_booked'] / df_merged_q4['total_seats']).clip(upper=1.0).fillna(0)

load_delay_correlation = df_merged_q4['load_factor'].corr(df_merged_q4['departure_delay_minutes'])

q3_load = df_merged_q4['load_factor'].quantile(0.75)
q1_load = df_merged_q4['load_factor'].quantile(0.25)

high_load_delay = df_merged_q4[df_merged_q4['load_factor'] >= q3_load]['departure_delay_minutes'].mean()
low_load_delay = df_merged_q4[df_merged_q4['load_factor'] <= q1_load]['departure_delay_minutes'].mean()


# --- 5. Tabular Output ---
data = {
    'Metric': [
        'Correlation: Load Factor vs. Departure Delay',
        f'Avg Delay for High Load Flights (Load Factor $\\geq$ {q3_load:.2f})',
        f'Avg Delay for Low Load Flights (Load Factor $\\leq$ {q1_load:.2f})'
    ],
    'Value': [
        f"{load_delay_correlation:.4f}",
        f"{high_load_delay:.2f}",
        f"{low_load_delay:.2f}"
    ],
    'Unit/Context': [
        'Pearson R',
        'Minutes',
        'Minutes'
    ]
}

df_output_q4 = pd.DataFrame(data)

print("### EDA Q4: Passenger Load and Delay Correlation")
print(df_output_q4.to_markdown(index=False))

# Save the full merged dataframe for the next step (Q5)
df_merged_q4.to_csv('temp_df_merged_q4.csv', index=False)

### EDA Q4: Passenger Load and Delay Correlation
| Metric                                                    |   Value | Unit/Context   |
|:----------------------------------------------------------|--------:|:---------------|
| Correlation: Load Factor vs. Departure Delay              | -0.1763 | Pearson R      |
| Avg Delay for High Load Flights (Load Factor $\geq$ 0.99) | 14.48   | Minutes        |
| Avg Delay for Low Load Flights (Load Factor $\leq$ 0.85)  | 37.84   | Minutes        |


In [5]:
import pandas as pd
import numpy as np

# --- 1. Load Data ---
try:
    # Load the intermediate file saved from the previous step (Q4)
    df_merged_base = pd.read_csv('temp_df_merged_q4.csv')
    df_pnr_flight = pd.read_csv("PNR+Flight+Level+Data.csv")
    df_pnr_remark = pd.read_csv("PNR Remark Level Data.csv")
except Exception as e:
    print(f"Error loading files or intermediate dataframe: {e}")
    exit()

flight_key_cols = ['company_id', 'flight_number', 'scheduled_departure_date_local']
pnr_unique_cols = ['record_locator', 'flight_number', 'pnr_creation_date']

pnr_cols_to_select = list(set(pnr_unique_cols + flight_key_cols))

df_ssr_temp = df_pnr_remark.merge(
    df_pnr_flight[pnr_cols_to_select].drop_duplicates(),
    on=pnr_unique_cols,
    how='inner'
)

df_ssr_counts = df_ssr_temp.groupby(flight_key_cols).size().reset_index(name='total_ssr_count')

df_final = df_merged_base.merge(df_ssr_counts, on=flight_key_cols, how='left')
df_final['total_ssr_count'] = df_final['total_ssr_count'].fillna(0)

q3_load = df_final['load_factor'].quantile(0.75)
q1_load = df_final['load_factor'].quantile(0.25)

q3_ssr = df_final['total_ssr_count'].quantile(0.75)
q1_ssr = df_final['total_ssr_count'].quantile(0.25)

mid_load_df = df_final[
    (df_final['load_factor'] > q1_load) &
    (df_final['load_factor'] < q3_load)
]

high_ssr_delay_mid_load = mid_load_df[mid_load_df['total_ssr_count'] >= q3_ssr]['departure_delay_minutes'].mean()
low_ssr_delay_mid_load = mid_load_df[mid_load_df['total_ssr_count'] <= q1_ssr]['departure_delay_minutes'].mean()

# --- 4. Tabular Output ---
data = {
    'Metric': [
        'Avg Delay for High SSR Flights (Mid Load)',
        'Avg Delay for Low SSR Flights (Mid Load)',
        'SSR Q1 for Low SSR Classification',
        'SSR Q3 for High SSR Classification'
    ],
    'Value': [
        f"{high_ssr_delay_mid_load:.2f}",
        f"{low_ssr_delay_mid_load:.2f}",
        f"{q1_ssr:.1f}",
        f"{q3_ssr:.1f}"
    ],
    'Unit/Context': [
        'Minutes',
        'Minutes',
        'Total SSRs/Flight',
        'Total SSRs/Flight'
    ]
}

df_output_q5 = pd.DataFrame(data)

print("### EDA Q5: SSR and Delay Analysis (Controlling for Load)")
print(df_output_q5.to_markdown(index=False))

# Save the final merged dataframe for the next deliverable (Difficulty Score)
df_final.to_csv('final_merged_analysis_data.csv', index=False)

### EDA Q5: SSR and Delay Analysis (Controlling for Load)
| Metric                                    |   Value | Unit/Context      |
|:------------------------------------------|--------:|:------------------|
| Avg Delay for High SSR Flights (Mid Load) |   20.63 | Minutes           |
| Avg Delay for Low SSR Flights (Mid Load)  |   13.72 | Minutes           |
| SSR Q1 for Low SSR Classification         |    1    | Total SSRs/Flight |
| SSR Q3 for High SSR Classification        |    3    | Total SSRs/Flight |


## Flight Difficulty Score Development

In [6]:
import pandas as pd
import numpy as np

# --- 1. Load All Original Data Files ---
print("--- Starting Robust Composite Index Score Generation (Final Fix) ---")
try:
    df_flight = pd.read_csv("Flight Level Data.csv")
    df_pnr_flight = pd.read_csv("PNR+Flight+Level+Data.csv")
    df_pnr_remark = pd.read_csv("PNR Remark Level Data.csv")
    df_bag = pd.read_csv("Bag+Level+Data.csv")
except Exception as e:
    print(f"FATAL ERROR: One of the required raw data files is missing ({e}). Cannot proceed.")
    exit()

# Define key column set
flight_key_cols = ['company_id', 'flight_number', 'scheduled_departure_date_local']
pnr_unique_cols = ['record_locator', 'flight_number', 'pnr_creation_date']

# --- 2. Recalculate and Merge All 3 Core Features ---

# A. Ground Time Risk Feature (from df_flight)
# CORRECTION: Include 'scheduled_arrival_station_code' here for the final output
core_cols = flight_key_cols + ['scheduled_arrival_station_code', 'scheduled_ground_time_minutes', 'minimum_turn_minutes', 'total_seats']
df_features = df_flight[core_cols].copy()

df_features['tight_ground_time'] = np.maximum(0, df_features['minimum_turn_minutes'] - df_features['scheduled_ground_time_minutes'])


# B. Baggage Complexity Feature (from df_bag)
df_bag['transfer_ind'] = df_bag['bag_type'].apply(lambda x: 1 if x in ['Transfer', 'Hot Transfer'] else 0)
df_bag['origin_ind'] = df_bag['bag_type'].apply(lambda x: 1 if x == 'Origin' else 0)
df_bag_agg = df_bag.groupby(flight_key_cols).agg(
    transfer_bags=('transfer_ind', 'sum'),
    origin_bags=('origin_ind', 'sum')
).reset_index()

df_features = df_features.merge(df_bag_agg, on=flight_key_cols, how='left')
df_features[['transfer_bags', 'origin_bags']] = df_features[['transfer_bags', 'origin_bags']].fillna(0)
df_features['total_bags_per_seat'] = (df_features['transfer_bags'] + df_features['origin_bags']) / df_features['total_seats'].replace(0, 1)
df_features['total_bags_per_seat'] = df_features['total_bags_per_seat'].replace([np.inf, -np.inf], 0).fillna(0)


# C. Customer/SSR Complexity Feature (from df_pnr_flight & df_pnr_remark)
# Recalculate Total Pax and SSRs (Logic remains the same)
pax_key_cols = flight_key_cols + ['record_locator']
df_pax = df_pnr_flight.groupby(pax_key_cols)['total_pax'].max().reset_index()
df_total_pax = df_pax.groupby(flight_key_cols)['total_pax'].sum().reset_index(name='total_pax_booked')

ssr_cols_to_select = list(set(pnr_unique_cols + flight_key_cols))
df_ssr_temp = df_pnr_remark.merge(
    df_pnr_flight[ssr_cols_to_select].drop_duplicates(),
    on=pnr_unique_cols,
    how='inner'
)
df_ssr_counts = df_ssr_temp.groupby(flight_key_cols).size().reset_index(name='total_ssr_count')

# Merge Pax and SSR counts
df_features = df_features.merge(df_total_pax, on=flight_key_cols, how='left')
df_features = df_features.merge(df_ssr_counts, on=flight_key_cols, how='left')

df_features['total_pax_booked'] = df_features['total_pax_booked'].fillna(0)
df_features['total_ssr_count'] = df_features['total_ssr_count'].fillna(0)
df_features['ssr_per_pax'] = df_features['total_ssr_count'] / df_features['total_pax_booked'].replace(0, 1)
df_features['ssr_per_pax'] = df_features['ssr_per_pax'].replace([np.inf, -np.inf], 0).fillna(0)

score_features = ['tight_ground_time', 'total_bags_per_seat', 'ssr_per_pax']
 

# --- 3. Daily Normalization and Score Calculation ---

def daily_min_max_scale(df, features):
    for col in features:
        min_val = df[col].min()
        max_val = df[col].max()
        df[f'{col}_norm'] = (df[col] - min_val) / (max_val - min_val) if (max_val - min_val) != 0 else 0
    return df

# Apply daily normalization
df_final_scored = df_features.groupby('scheduled_departure_date_local', group_keys=False).apply(daily_min_max_scale, score_features)

# Calculate Difficulty Score (Equal Weights)
df_final_scored['difficulty_score'] = (
    df_final_scored['tight_ground_time_norm'] +
    df_final_scored['total_bags_per_seat_norm'] +
    df_final_scored['ssr_per_pax_norm']
) / len(score_features)


# --- 4. Daily Ranking & Classification ---

def classify_difficulty(df):
    df['daily_difficulty_rank'] = df['difficulty_score'].rank(method='min', ascending=False).astype(int)
    q_25 = df['difficulty_score'].quantile(0.25)
    q_75 = df['difficulty_score'].quantile(0.75)
    def classify(score):
        if score >= q_75: return 'Difficult'
        elif score > q_25: return 'Medium'
        else: return 'Easy'
    df['difficulty_classification'] = df['difficulty_score'].apply(classify)
    return df

df_final_scored = df_final_scored.groupby('scheduled_departure_date_local', group_keys=False).apply(classify_difficulty)

df_deliverable = df_final_scored[[
    'scheduled_departure_date_local', 'flight_number', 'scheduled_arrival_station_code',
    'difficulty_score', 'daily_difficulty_rank', 'difficulty_classification'
]].sort_values(by=['scheduled_departure_date_local', 'daily_difficulty_rank'])

print("### Deliverable 2: Flight Difficulty Score (Composite Index) - Top 10 Ranked Flights")
print(df_deliverable.head(10).to_markdown(index=False, floatfmt=".2f"))

# Save the final scored dataframe for post-analysis
df_final_scored.to_csv("flight_difficulty_score_deliverable.csv", index=False)
print("\nFile 'flight_difficulty_score_deliverable.csv' saved successfully.")

--- Starting Robust Composite Index Score Generation (Final Fix) ---


C:\Users\HomePC\AppData\Local\Temp\ipykernel_2972\3161204042.py:79: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final_scored = df_features.groupby('scheduled_departure_date_local', group_keys=False).apply(daily_min_max_scale, score_features)
C:\Users\HomePC\AppData\Local\Temp\ipykernel_2972\3161204042.py:102: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final_scored = df_final_scored.groupby('scheduled_depar

### Deliverable 2: Flight Difficulty Score (Composite Index) - Top 10 Ranked Flights
| scheduled_departure_date_local   |   flight_number | scheduled_arrival_station_code   |   difficulty_score |   daily_difficulty_rank | difficulty_classification   |
|:---------------------------------|----------------:|:---------------------------------|-------------------:|------------------------:|:----------------------------|
| 2025-08-01                       |            5376 | LEX                              |               0.67 |                       1 | Difficult                   |
| 2025-08-01                       |            1811 | MSP                              |               0.55 |                       2 | Difficult                   |
| 2025-08-01                       |            4425 | GRR                              |               0.53 |                       3 | Difficult                   |
| 2025-08-01                       |            5603 | FNT                      